### Imports

In [1]:
import numpy as np 
import pandas as pd
import requests  
import pycountry 
from google.oauth2 import service_account
from googleapiclient.discovery import build
from functions import *
from conf.settings import *
import re
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Data Extraction and Casting

In [2]:
df_metrics_by_age = get_named_range_as_dataframe(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, AGE_NAMED_RANGE)
df_metrics_by_age = convert_column_names_to_standard_english(df_metrics_by_age)
df_metrics_by_age = clean_and_cast_columns(df_metrics_by_age)

In [28]:
df_metrics_by_age.columns

Index(['ad_set_name', 'age', 'result_rate', 'result_indicator', 'results',
       'reach', 'frequency', 'links_clicks', 'cpc_link', 'cpc_general',
       'cost_per_1000_people_reached', 'ctr', 'add_to_cart',
       'cost_per_add_to_cart', 'initiate_checkout',
       'cost_per_initiate_checkout', 'purchase', 'cost_per_purchase',
       'amount_spent', 'purchase_conversion_value', 'campaign_goal',
       'classified_campaign_goal', 'classified_result_indicator'],
      dtype='object')

In [36]:
df_teste = (df_metrics_by_age.groupby(["age"]).agg({            
                                        "cpc_link": "mean",                     
                                        "cpc_general": "mean",                         
                                        }).reset_index())

df_teste.round(2)

,age,cpc_link,cpc_general
0,13-17,0.09,0.03
1,18-24,0.50,0.12
2,25-34,0.79,0.16
3,35-44,0.45,0.19
4,45-54,0.21,0.13
5,55-64,0.20,0.13
6,65+,0.11,0.07


### Analysis

In [19]:
print(df_metrics_by_age.query('classified_campaign_goal == "Engajamento/Visualização" and age == "13-17"').cpc_general.mean())
print(df_metrics_by_age.query('classified_campaign_goal == "Engajamento/Visualização" and age == "18-24"').cpc_general.mean())
print(df_metrics_by_age.query('classified_campaign_goal == "Engajamento/Visualização" and age == "25-34"').cpc_general.mean())
print(df_metrics_by_age.query('classified_campaign_goal == "Engajamento/Visualização" and age == "35-44"').cpc_general.mean())
print(df_metrics_by_age.query('classified_campaign_goal == "Conversão"').cpc_general.mean())

0.007954545454545454
0.2159848484848485
0.2678787878787879
0.18030534351145036
0.12379644165358451


In [3]:
df_metrics_by_age['campaign_goal'] = df_metrics_by_age['ad_set_name'].apply(create_column_campaign_goal)
df_metrics_by_age['classified_campaign_goal'] = df_metrics_by_age['campaign_goal'].apply(classify_campaign)
df_metrics_by_age['classified_result_indicator'] = df_metrics_by_age['result_indicator'].apply(lambda x: 'pixel_initiate_checkout' if 'fb_pixel_initiate_checkout' in x else  
                                                                                                         'pixel_add_to_cart' if 'fb_pixel_add_to_cart' in x else              
                                                                                                         'pixel_purchase' if 'fb_pixel_purchase' in x else                    
                                                                                                         'post_engagement' if 'post_engagement' in x else                     
                                                                                                         'Outros')

In [4]:
df_metrics_by_age.columns

Index(['ad_set_name', 'age', 'result_rate', 'result_indicator', 'results',
       'reach', 'frequency', 'links_clicks', 'cpc_link', 'cpc_general',
       'cost_per_1000_people_reached', 'ctr', 'add_to_cart',
       'cost_per_add_to_cart', 'initiate_checkout',
       'cost_per_initiate_checkout', 'purchase', 'cost_per_purchase',
       'amount_spent', 'purchase_conversion_value', 'campaign_goal',
       'classified_campaign_goal', 'classified_result_indicator'],
      dtype='object')

In [5]:
print(df_metrics_by_age['classified_campaign_goal'].value_counts())
print(df_metrics_by_age['age'].value_counts())
print(df_metrics_by_age['result_indicator'].value_counts())
print(df_metrics_by_age[['ad_set_name','purchase','reach']].sort_values(by='purchase', ascending=False).head(10))


classified_campaign_goal
Conversão                   3822
Engajamento/Visualização     806
Outros                        19
Name: count, dtype: int64
age
25-34    1088
18-24    1081
35-44    1074
13-17     546
45-54     362
65+       260
55-64     236
Name: count, dtype: int64
result_indicator
-                                                        1902
actions:offsite_conversion.fb_pixel_initiate_checkout     995
actions:offsite_conversion.fb_pixel_add_to_cart           790
actions:offsite_conversion                                452
actions:offsite_conversion.fb_pixel_purchase              309
actions:post_engagement                                   182
actions:link_click                                          9
estimated_ad_recallers                                      8
Name: count, dtype: int64
                                                      ad_set_name  purchase  \
2994                   LC Checkout-Conversion 15/10/2016 (WW, GA)     152.0   
2995                     

In [6]:
age_grouped = (df_metrics_by_age.groupby(["age", "classified_campaign_goal", "classified_result_indicator"]).agg({"results": "sum",                      
                                                                                                                  "result_rate": "mean",      
                                                                                                                  "reach": "sum",                     
                                                                                                                  "frequency": "mean",             
                                                                                                                  "cpc_link": "mean",                     
                                                                                                                  "cpc_general": "mean",                   
                                                                                                                  "cost_per_1000_people_reached": "mean",       
                                                                                                                  "ctr": "mean",                          
                                                                                                                  "add_to_cart": "sum",            
                                                                                                                  "cost_per_add_to_cart": "mean",           
                                                                                                                  "initiate_checkout": "sum",                    
                                                                                                                  "cost_per_initiate_checkout": "mean",        
                                                                                                                  "purchase": "sum",                 
                                                                                                                  "cost_per_purchase": "mean",             
                                                                                                                  "amount_spent": "sum",                       
                                                                                                                  "purchase_conversion_value": "sum",
                                                                                                                  "links_clicks": "sum",       
                                                                                                                  }).reset_index()
                                                                                                                  .rename(columns={'ad_set_name': 'qty_ad_sets'}))

age_grouped["roi"] = age_grouped["purchase_conversion_value"] / age_grouped["amount_spent"]
age_grouped["cart_to_checkout_conversion"] = np.where(age_grouped["add_to_cart"] > 0, 
                                                      age_grouped["initiate_checkout"] / age_grouped["add_to_cart"], 
                                                      0)
age_grouped["checkout_to_purchase_conversion"] = np.where(age_grouped["initiate_checkout"] > 0,  
                                                          age_grouped["purchase"] / age_grouped["initiate_checkout"], 
                                                          0)

In [7]:
age_grouped.head()

,age,classified_campaign_goal,classified_result_indicator,results,result_rate,reach,frequency,cpc_link,cpc_general,cost_per_1000_people_reached,ctr,add_to_cart,cost_per_add_to_cart,initiate_checkout,cost_per_initiate_checkout,purchase,cost_per_purchase,amount_spent,purchase_conversion_value,links_clicks,roi,cart_to_checkout_conversion,checkout_to_purchase_conversion
0,13-17,Conversão,Outros,0,0.000000,268191.0,0.296228,0.049550,0.015779,0.414706,0.033335,175,0.265398,17,0.065675,0.0,0.000000,307.94,0.00,2836,0.000000,0.097143,0.000000
1,13-17,Conversão,pixel_add_to_cart,15959,0.001141,7070261.0,1.487955,0.182727,0.061250,1.626477,0.021914,15959,1.650227,1993,8.890114,296.0,26.540909,10948.83,9397.31,112939,0.858294,0.124883,0.148520
2,13-17,Conversão,pixel_initiate_checkout,3612,0.000165,10788231.0,1.642655,0.140708,0.047345,1.285841,0.024350,23607,1.251947,3612,6.387168,551.0,21.802655,13748.12,16907.81,174986,1.229827,0.153005,0.152547
3,13-17,Conversão,pixel_purchase,127,0.000060,2126351.0,1.869000,0.114000,0.039000,1.676000,0.026290,6552,0.646000,800,6.594000,127.0,46.810000,3693.38,3945.49,40055,1.068260,0.122100,0.158750
4,13-17,Engajamento/Visualização,Outros,3,0.062500,24.0,0.291667,0.000417,0.000417,0.208333,0.041667,0,0.000000,0,0.000000,0.0,0.000000,0.08,0.00,1,0.000000,0.000000,0.000000


In [8]:
print("Aggregated Metrics by Age Group:")
age_grouped = convert_column_standard_english_to_portuguse(age_grouped, base = 'EN')

Aggregated Metrics by Age Group:


### Save to sheets

In [ ]:
column_order = ["idade",
                "objetivo_classificado_da_campanha",
                "classified_result_indicator",
                "resultados",
                "taxa_de_resultado",
                "valor_gasto",
                "alcance",
                "compra",
                "valor_conversao_compra",
                "roi",
                "frequencia",
                "custo_por_1000_pessoas_alcancadas",
                "cliques_no_link",
                "custo_por_clique_no_link",
                "custo_por_clique_geral",
                "ctr",
                "adicionar_ao_carrinho",
                "custo_por_adicionar_ao_carrinho",
                "iniciar_finalizacao_da_compra",
                "custo_por_iniciar_finalizacao_da_compra",
                "custo_por_compra",
                "conversao_carrinho_checkout",
                "conversao_checkout_compra",]
age_grouped = age_grouped[column_order]
age_grouped = convert_column_standard_english_to_portuguse(age_grouped, base = 'PT')
#save_df_to_named_range(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, 'df_w_metrics_by_age', age_grouped)

In [10]:
age_grouped.head()

,age,classified_campaign_goal,classified_result_indicator,results,result_rate,amount_spent,reach,purchase,purchase_conversion_value,roi,frequency,cost_per_1000_people_reached,links_clicks,cpc_link,cpc_general,ctr,add_to_cart,cost_per_add_to_cart,initiate_checkout,cost_per_initiate_checkout,cost_per_purchase,cart_to_checkout_conversion,checkout_to_purchase_conversion
0,13-17,Conversão,Outros,0,0.000000,307.94,268191.0,0.0,0.00,0.000000,0.296228,0.414706,2836,0.049550,0.015779,0.033335,175,0.265398,17,0.065675,0.000000,0.097143,0.000000
1,13-17,Conversão,pixel_add_to_cart,15959,0.001141,10948.83,7070261.0,296.0,9397.31,0.858294,1.487955,1.626477,112939,0.182727,0.061250,0.021914,15959,1.650227,1993,8.890114,26.540909,0.124883,0.148520
2,13-17,Conversão,pixel_initiate_checkout,3612,0.000165,13748.12,10788231.0,551.0,16907.81,1.229827,1.642655,1.285841,174986,0.140708,0.047345,0.024350,23607,1.251947,3612,6.387168,21.802655,0.153005,0.152547
3,13-17,Conversão,pixel_purchase,127,0.000060,3693.38,2126351.0,127.0,3945.49,1.068260,1.869000,1.676000,40055,0.114000,0.039000,0.026290,6552,0.646000,800,6.594000,46.810000,0.122100,0.158750
4,13-17,Engajamento/Visualização,Outros,3,0.062500,0.08,24.0,0.0,0.00,0.000000,0.291667,0.208333,1,0.000417,0.000417,0.041667,0,0.000000,0,0.000000,0.000000,0.000000,0.000000


### Métricas de conversão e engajamento


In [11]:
df_metricas_by_age_engajamento = calcular_percentuais_por_objetivo(df_metrics_by_age, 'age', 'Engajamento/Visualização')
#save_df_to_named_range(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, 'df_analysis_by_age_engagement', df_metricas_by_age_engajamento)
df_metricas_by_age_engajamento

,age,classified_campaign_goal,results,purchase,purchase_conversion_value,amount_spent,count,pct_results,pct_purchase,pct_purchase_conversion_value,pct_amount_spent,pct_count
0,13-17,Engajamento/Visualização,366577,1.0,24.90,325.18,44,0.12,0.00,0.00,0.03,0.05
1,18-24,Engajamento/Visualização,1914307,498.0,16446.37,6328.66,132,0.64,0.51,0.49,0.55,0.16
2,25-34,Engajamento/Visualização,432590,326.0,11792.89,3308.37,132,0.15,0.33,0.35,0.29,0.16
3,35-44,Engajamento/Visualização,151404,111.0,3793.95,1133.53,131,0.05,0.11,0.11,0.10,0.16
4,45-54,Engajamento/Visualização,58497,37.0,1258.18,290.30,132,0.02,0.04,0.04,0.03,0.16
5,55-64,Engajamento/Visualização,16876,11.0,323.45,60.52,117,0.01,0.01,0.01,0.01,0.15
6,65+,Engajamento/Visualização,37074,2.0,42.98,48.11,118,0.01,0.00,0.00,0.00,0.15


In [ ]:
df_metricas_by_age_conversao = calcular_percentuais_por_objetivo(df_metrics_by_age, 'age')
#save_df_to_named_range(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, 'df_analysis_by_age_conversion', df_metricas_by_age_conversao)
df_metricas_by_age_conversao

,age,classified_campaign_goal,results,purchase,purchase_conversion_value,amount_spent,count,pct_results,pct_purchase,pct_purchase_conversion_value,pct_amount_spent,pct_count
0,13-17,Conversão,19698,974.0,30250.61,28698.27,500,0.25,0.11,0.11,0.14,0.13
1,18-24,Conversão,34513,3647.0,116425.46,92324.64,943,0.44,0.43,0.43,0.46,0.25
2,25-34,Conversão,19481,3226.0,103279.80,65849.27,950,0.25,0.38,0.38,0.33,0.25
3,35-44,Conversão,4473,579.0,18788.01,12461.42,938,0.06,0.07,0.07,0.06,0.25
4,45-54,Conversão,203,37.0,1211.15,796.77,230,0.0,0.00,0.00,0.00,0.06
5,55-64,Conversão,46,9.0,251.22,172.70,119,0.0,0.00,0.00,0.00,0.03
6,65+,Conversão,22,4.0,146.30,137.45,142,0.0,0.00,0.00,0.00,0.04
